In [7]:
import pyzx as zx
from tree_decompose import *
import sys
sys.path.append('..')
from Graph import *
import os
from qiskit import transpile, QuantumCircuit
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
# directory = '../../flow-pyzx/circuits/Fast/'
directory = 'QASMBench/small/'
for circuit in os.listdir(directory):
    print(circuit)
    try:
        qc = QuantumCircuit.from_qasm_file(os.path.join(directory,circuit+'/'+circuit+'.qasm'))
        qc = transpile(qc, basis_gates=['id','cx', 'cz', 'rz','rx', 'h'])

        c = zx.Circuit.from_qasm(qc.qasm())
    except:
        print("error",circuit)
        continue
#     c = zx.Circuit.load(os.path.join(directory,circuit+'/'+circuit+'.qasm'))
#     g = to_gl(c.to_graph())
    g = c.to_graph()
    if g.num_vertices() > 1000:
        print("skip ",circuit)
        continue
    optimize_graph(g)
    minputs = [list(g.neighbors(input))[0] for input in g.inputs()]
    moutputs = [list(g.neighbors(output))[0] for output in g.outputs()]
    g1 = g.clone()
    g1.remove_vertices(set(g.inputs()).union(set(g.outputs())))
    if g1.num_vertices() > 100:
        print("skip ",circuit)
        continue
    try:
        if is_totally_decomposable(g1):
            print(circuit,"is tree")
        else:
            splits = standard_decomposition(g1)
            print(circuit,"has ",len(splits),"splits")
    except:
        print("error prime decomposition")
#     zx.simplify.interior_clifford_simp(g, quiet=True)
    
#     minputs = [list(g.neighbors(input))[0] for input in g.inputs()]
#     moutputs = [list(g.neighbors(output))[0] for output in g.outputs()]
#     gp = g.clone()
#     gp.remove_vertices(set(g.inputs()).union(set(g.outputs())))
#     if g.num_vertices() > 100:
# #         print("skip ",circuit)
#         continue
#     print("circuit",circuit)
#     gp, complementations = graph_like_to_tree(g)
#     if complementations:
#         print(circuit,complementations,"saved",g.num_edges(),gp.num_edges())
    
#     splits = prime_decomposition(gp)
#     if len(splits) > 1:
#         print("vertices",gp.num_vertices(),"number of splits",len(splits))
#     if len(splits) > 5:
#         zx.draw(gp, labels=True)
#         import pdb
#         pdb.set_trace()

basis_trotter_n4
error basis_trotter_n4
basis_change_n3
error basis_change_n3
teleportation_n3
teleportation_n3 is tree
iswap_n2
iswap_n2 is tree
pea_n5
pea_n5 has  13 splits
hhl_n7
skip  hhl_n7
grover_n2
error grover_n2
quantumwalks_n2
quantumwalks_n2 has  11 splits
shor_n5
error shor_n5
bb84_n8
bb84_n8 has  2 splits
vqe_uccsd_n4
error vqe_uccsd_n4
vqe_uccsd_n6
error vqe_uccsd_n6
vqe.py
error vqe.py
fredkin_n3
fredkin_n3 has  6 splits
bell_n4
bell_n4 is tree
toffoli_n3
toffoli_n3 has  8 splits
adder_n10
adder_n10 has  18 splits
dnn_n2
error prime decomposition
error_correctiond3_n5
error_correctiond3_n5 has  2 splits
lpn_n5
lpn_n5 is tree
linearsolver_n3
linearsolver_n3 is tree
qec_sm_n5
error qec_sm_n5
adder_n4
adder_n4 has  10 splits
cat_state_n4
cat_state_n4 is tree
qft_n4
qft_n4 has  11 splits
hhl_n10
error hhl_n10
wstate_n3
wstate_n3 has  11 splits
qec_en_n5
qec_en_n5 has  1 splits
hs4_n4
hs4_n4 has  4 splits
sat_n7
error prime decomposition
inverseqft_n4
error inverseqft_n4
qeli

In [2]:
c = zx.Circuit.load('QASMBench/small/qaoa_n6/qaoa_n6.qasm')
g = c.to_graph()
optimize_graph(g)
# zx.simplify.interior_clifford_simp(g, quiet=True)
minputs = [list(g.neighbors(input))[0] for input in g.inputs()]
moutputs = [list(g.neighbors(output))[0] for output in g.outputs()]
g1 = g.clone()
g1.remove_vertices(set(g.inputs()).union(set(g.outputs())))
zx.draw(g1, labels=True)

In [3]:
if is_totally_decomposable(g1):
    print("is tree")
else:
    splits = standard_decomposition(g1)
    print("has ",len(splits),"splits")

has  31 splits


In [4]:
print(splits)
zx.draw(g1, labels=True)

[{258, 259, 132, 401, 402, 27, 418, 708, 453, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 596, 723, 724, 725, 726, 345, 727, 728, 729, 730, 222, 97, 365}, {730, 603, 701}, {729, 731, 653}, {595, 731, 695}, {728, 532, 679}, {706, 462, 727}, {578, 732, 726}, {707, 732, 605}, {610, 651, 725}, {724, 507, 652}, {427, 699, 723}, {722, 579, 733}, {388, 733, 703}, {721, 683, 380}, {344, 672, 720}, {719, 676, 319}, {289, 211, 718}, {580, 717, 367}, {698, 139, 716}, {106, 715, 734}, {354, 685, 734}, {176, 714, 359}, {713, 700, 141}, {712, 682, 237}, {704, 282, 735}, {711, 735, 71}, {736, 689, 247}, {736, 36, 710}, {709, 131, 677}, {737, 26, 708}, {352, 737, 693}]


In [6]:
for split in splits:
    if len(split) == 3:
        g2 = g1.clone()
        g2.remove_vertices(set(g2.vertices()).difference(split))
        v2 = get_star_center(g2)
        print(split, is_star(g2), g2.type(v2)== VertexType.X)

{730, 603, 701} True False
{729, 731, 653} True True
{595, 731, 695} True False
{728, 532, 679} True False
{706, 462, 727} True False
{578, 732, 726} True True
{707, 732, 605} True False
{610, 651, 725} True False
{724, 507, 652} True False
{427, 699, 723} True False
{722, 579, 733} True True
{388, 733, 703} True False
{721, 683, 380} True False
{344, 672, 720} True False
{719, 676, 319} True False
{289, 211, 718} True False
{580, 717, 367} True False
{698, 139, 716} True False
{106, 715, 734} True True
{354, 685, 734} True False
{176, 714, 359} True False
{713, 700, 141} True False
{712, 682, 237} True False
{704, 282, 735} True False
{711, 735, 71} True True
{736, 689, 247} True False
{736, 36, 710} True True
{709, 131, 677} True False
{737, 26, 708} True True
{352, 737, 693} True False


In [5]:
complementations = correct_components(g1, splits)
recompose_graph(g1, splits)
zx.draw(g1, labels=True)

lc on  258
rec lc on  714
rec lc on  717
rec lc on  719
rec lc on  721
rec lc on  722
rec lc on  723
rec lc on  724
rec lc on  728
rec lc on  708
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  734
rec lc on  718
rec lc on  725
rec lc on  733
rec lc on  727
rec lc on  730
rec lc on  726
rec lc on  729
rec lc on  732
rec lc on  731
rec lc on  737
rec lc on  736
rec lc on  735


lc on  653
rec lc on  729
rec lc on  731
rec lc on  725
rec lc on  719
rec lc on  721
rec lc on  726
rec lc on  727
rec lc on  728
rec lc on  730
rec lc on  732
rec lc on  724
rec lc on  717
rec lc on  722
rec lc on  723
rec lc on  714
rec lc on  733
rec lc on  708
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  734
rec lc on  718
rec lc on  737
rec lc on  736
rec lc on  735


lc on  258
rec lc on  714
rec lc on  717
rec lc on  719
rec lc on  721
rec lc on  722
rec lc on  723
rec lc on  724
rec lc on  728
rec lc on  708
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  734
rec lc on  718
rec lc on  725
rec lc on  733
rec lc on  727
rec lc on  730
rec lc on  726
rec lc on  729
rec lc on  732
rec lc on  737
rec lc on  736
rec lc on  735


lc on  578
rec lc on  726
rec lc on  732
rec lc on  725
rec lc on  719
rec lc on  721
rec lc on  727
rec lc on  728
rec lc on  729
rec lc on  730
rec lc on  717
rec lc on  724
rec lc on  722
rec lc on  723
rec lc on  733


lc on  258
rec lc on  714
rec lc on  717
rec lc on  719
rec lc on  721
rec lc on  722
rec lc on  723
rec lc on  724
rec lc on  728
rec lc on  708
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  734
rec lc on  718
rec lc on  725
rec lc on  733
rec lc on  727
rec lc on  730
rec lc on  726
rec lc on  729
rec lc on  737
rec lc on  736
rec lc on  735


...

lc on  71
rec lc on  711
rec lc on  735
rec lc on  714
rec lc on  708
rec lc on  710
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  737
rec lc on  736
rec lc on  718


lc on  258
rec lc on  714
rec lc on  717
rec lc on  719
rec lc on  721
rec lc on  722
rec lc on  723
rec lc on  724
rec lc on  728
rec lc on  708
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  718
rec lc on  725
rec lc on  727
rec lc on  730
rec lc on  726
rec lc on  729
rec lc on  737
rec lc on  736


lc on  36
rec lc on  710
rec lc on  736
rec lc on  714
rec lc on  708
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  717
rec lc on  737
rec lc on  718
rec lc on  719
rec lc on  721
rec lc on  722
rec lc on  723
rec lc on  724
rec lc on  728
rec lc on  725
rec lc on  727
rec lc on  730
rec lc on  726
rec lc on  729


lc on  258
rec lc on  714
rec lc on  717
rec lc on  719
rec lc on  721
rec lc on  722
rec lc on  723
rec lc on  724
rec lc on  728
rec lc on  708
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  718
rec lc on  725
rec lc on  727
rec lc on  730
rec lc on  726
rec lc on  729
rec lc on  737


lc on  26
rec lc on  708
rec lc on  737
rec lc on  714
rec lc on  710
rec lc on  711
rec lc on  712
rec lc on  715
rec lc on  716
rec lc on  720
rec lc on  718


In [7]:
g.num_edges(),g1.num_edges()

(157, 100)

In [6]:
gc = g.clone()
gc.remove_vertices(set(g.inputs()).union(set(g.outputs())))
for complementation in complementations:
    local_complement(gc, complementation, [])

lc on  258


lc on  653


lc on  258


lc on  578


lc on  258


...

lc on  71


lc on  258


lc on  36


lc on  258


lc on  26


In [9]:
gc.num_edges()

140

In [3]:
splits = standard_decomposition(g1)
print(splits)
zx.draw(g1, labels=True)

[{64, 100, 96}, {65, 100, 102}, {50, 102, 103}, {4, 101, 103}, {98, 99, 106}, {26, 107, 108}, {35, 106, 107, 22}, {108, 11, 12, 101}]


In [5]:
gt, lcs = graph_like_to_tree(g)
zx.draw(gt, labels=True)

In [3]:
# c = zx.Circuit.load('../../flow-pyzx/circuits/Fast/tof_4_after_heavy')
c = zx.Circuit.load('QASMBench/small/bell_n4/bell_n4.qasm')
# g = to_gl(c.to_graph())
g = c.to_graph()
zx.simplify.interior_clifford_simp(g, quiet=True)
gp, complementations = graph_like_to_tree(g)
# minputs = [list(g.neighbors(input))[0] for input in g.inputs()]
# moutputs = [list(g.neighbors(output))[0] for output in g.outputs()]
# gp = g.clone()
# gp.remove_vertices(set(g.inputs()).union(set(g.outputs())))
# splits = prime_decomposition(gp)
# print(splits)
gp.normalize()
zx.draw(gp, labels=True)

In [6]:
is_totally_decomposable(g)

True

In [4]:
zx.draw(g, labels=True)

In [16]:
inputs = []
for minput in minputs:
    input = gp.add_vertex()
    print("newv",input)
    gp.add_edge(gp.edge(input, minput))
    inputs.append(input)
gp.set_inputs(inputs)

outputs = []
for moutput in moutputs:
    output = gp.add_vertex()
    print("newv",output)
    gp.add_edge(gp.edge(moutput, output))
    outputs.append(output)
gp.set_outputs(outputs)

gp.normalize()
zx.draw(gp, labels=True)

newv 88
newv 89
newv 90
newv 91
newv 92
newv 93
newv 94
newv 95
newv 96
newv 97
newv 98
newv 99
newv 100
newv 101


In [12]:
g.num_edges()

80

In [5]:
correct_components(gp, splits)
zx.draw(gp, labels=True)

component {128, 134, 9, 10, 11, 12, 140, 17, 18, 19, 146, 23, 24, 152, 29, 30, 31, 158, 35, 36, 164, 40, 41, 168, 169, 170, 171, 46, 47, 48, 172, 173, 174, 52, 53, 57, 58, 62, 63, 68, 69, 70, 74, 75, 79, 80, 84, 85, 89, 90, 95, 96, 97, 101, 102, 106, 107, 111, 112, 116, 117, 121, 122, 127}
component {128, 134, 9, 10, 11, 12, 140, 17, 18, 19, 146, 23, 24, 152, 29, 30, 31, 158, 35, 36, 164, 40, 41, 168, 169, 170, 171, 46, 47, 48, 172, 173, 174, 52, 53, 57, 58, 62, 63, 68, 69, 70, 74, 75, 79, 80, 84, 85, 89, 90, 95, 96, 97, 101, 102, 106, 107, 111, 112, 116, 117, 121, 122, 127} is not correctable
component {165, 174, 167}
component {161, 173, 159}
component {153, 155, 172}
component {171, 147, 149}
component {170, 141, 143}
component {137, 169, 135}
component {168, 129, 131}


In [3]:
g1, complementations = graph_like_to_tree(g)
print(complementations)
g1.normalize()
zx.draw(g1, labels=True)

component {5, 110, 7}
component {10, 11, 12, 13, 14, 15, 17, 18, 21, 22, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 63, 64, 65, 67, 68, 71, 72, 75, 76, 78, 79, 80, 81, 82, 84, 85, 86, 88, 89, 91, 92, 94, 95, 98, 99, 102, 103, 105, 106, 107, 108, 109, 112}
component {10, 11, 12, 13, 14, 15, 17, 18, 21, 22, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 63, 64, 65, 67, 68, 71, 72, 75, 76, 78, 79, 80, 81, 82, 84, 85, 86, 88, 89, 91, 92, 94, 95, 98, 99, 102, 103, 105, 106, 107, 108, 109, 112} is not correctable
component {112, 9, 111}
component {8, 110, 111}
[]


In [13]:
minputs = [list(g.neighbors(input))[0] for input in g.inputs()]
moutputs = [list(g.neighbors(output))[0] for output in g.outputs()]
gp = g.clone()
# gp.remove_vertices(set(g.inputs()).union(set(g.outputs())))
# res = prime_decomposition(gp)
# print("split components:",res)
zx.draw(gp, labels=True)

In [11]:
zx.draw(g, labels=True)

In [14]:
res = prime_decomposition(gp)
print("split components:",res)
zx.draw(gp, labels=True)

split components: [{0, 120, 117}, {1, 123, 63}, {11, 12, 14, 15, 17, 18, 21, 22, 25, 26, 32, 33, 34, 35, 37, 38, 40, 41, 44, 45, 48, 59, 61, 62, 64, 65, 67, 68, 71, 72, 75, 86, 88, 89, 91, 92, 94, 95, 99, 102, 118, 119, 120, 121, 122, 123, 124, 125, 126}, {113, 126, 103}, {98, 116, 125}, {124, 114, 76}, {49, 122, 115}, {121, 2, 36}, {9, 4, 119}, {10, 3, 118}, {112, 117, 13}]
